In [ ]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import os
import requests
from yt_dlp import YoutubeDL
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC
from PIL import Image
from io import BytesIO

# ---------------------------------------------------
# USER INPUT (multiple playlists supported)
# ---------------------------------------------------

raw_input = input("Paste YouTube playlist URLs (comma-separated):\n")
playlist_urls = [x.strip() for x in raw_input.split(",") if x.strip()]

music_base_folder = r"C:\Users\Philip\Music"

'''
# ---------------------------------------------------
# FUNCTION: Create folder "Artist - Album (Year)"
# ---------------------------------------------------
def create_target_folder(info):
    artist = info.get("artist") or info.get("channel") or "Unknown Artist"
    title = info.get("title") or "Unknown Album"
    year = info.get("release_year") or ""

    folder_name = f"{artist} - {title}"
    if year:
        folder_name += f" ({year})"

    target_folder = os.path.join(music_base_folder, folder_name)
    os.makedirs(target_folder, exist_ok=True)
    return target_folder
'''
# ---------------------------------------------------
# FUNCTION: Create folder "Artist - Album (Year)"
# ---------------------------------------------------
def create_target_folder(info):
    # Use artist if available
    artist = info.get("artist")
    
    # Fall back to uploader / channel name
    if not artist:
        artist = info.get("uploader") or info.get("uploader_id") or "Unknown Artist"
        artist = artist.replace("@", "").replace(" ", "").lower()  # Clean it up
    
    # Album title
    title = info.get("title") or "Unknown Album"
    
    # Year if available
    year = info.get("release_year") or ""
    
    # Build folder name
    folder_name = f"{artist} - {title}"
    if year:
        folder_name += f" ({year})"
    
    target_folder = os.path.join(music_base_folder, folder_name)
    os.makedirs(target_folder, exist_ok=True)
    return target_folder

# ---------------------------------------------------
# FUNCTION: Download playlist thumbnail as JPG
# ---------------------------------------------------
def download_playlist_thumbnail(info, target_folder):
    url = info.get("thumbnail")
    if not url:
        print("No playlist thumbnail found.")
        return None

    print("Downloading playlist artwork...")

    img_data = requests.get(url).content
    img = Image.open(BytesIO(img_data)).convert("RGB")

    cover_path = os.path.join(target_folder, "cover.jpg")
    img.save(cover_path, "JPEG", quality=95)

    print(f"Saved cover: {cover_path}")
    return cover_path


# ---------------------------------------------------
# FUNCTION: Embed cover into MP3
# ---------------------------------------------------
def embed_cover(mp3_file, cover_path):
    audio = MP3(mp3_file, ID3=ID3)
    try:
        audio.add_tags()
    except:
        pass

    audio.tags["APIC"] = APIC(
        mime="image/jpeg",
        type=3,
        desc="Cover",
        data=open(cover_path, "rb").read()
    )
    audio.save()


# ---------------------------------------------------
# MAIN PROCESSING LOOP
# ---------------------------------------------------
for playlist_url in playlist_urls:

    print("\n==============================================")
    print(f"Processing playlist: {playlist_url}")
    print("==============================================\n")

    # Step 1 — Retrieve playlist metadata only
    print("Fetching playlist information...")
    with YoutubeDL({"extract_flat": True}) as ydl:
        info = ydl.extract_info(playlist_url, download=False)

    # Step 2 — Make folder
    target_folder = create_target_folder(info)
    print("Saving to folder:", target_folder)

    # Step 3 — Download playlist cover
    cover_path = download_playlist_thumbnail(info, target_folder)

    # Step 4 — Download and convert tracks
    print("Downloading audio tracks and converting to MP3...")

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(target_folder, "%(playlist_index)02d - %(title)s.%(ext)s"),
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "320",
            }
        ],
        "addmetadata": True,
        "extractor_args": {"youtube": {"player_client": "default"}},
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([playlist_url])

    # Step 5 — Embed cover in all MP3s
    print("Embedding playlist cover art into MP3 files...")

    if cover_path:
        for filename in os.listdir(target_folder):
            if filename.lower().endswith(".mp3"):
                fullpath = os.path.join(target_folder, filename)
                embed_cover(fullpath, cover_path)

    print("Finished:", target_folder)

print("\nAll playlists processed successfully!")

In [ ]:
'''
# consider adding this to explicitly tell yt-dlp to use Node.js in Python if needed:


ydl_opts = {
    "extractor_args": {"youtube": {"player_client": "default"}},
    "postprocessors": [...],
    "outtmpl": "...",
    "format": "bestaudio/best",
    "nodejs": True  # this tells yt-dlp to use Node.js
}
'''